# 6. SWATH

This notebook shows how to run the SWATH controller on a ViMMS dataset

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
from pathlib import Path
import os

In [5]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SWATH
from vimms.Environment import Environment
from vimms.Common import POSITIVE, load_obj, set_log_level_warning, set_log_level_debug

### Load the data

Load pre-processed chemical data. Refer to [this notebook](https://github.com/glasgowcompbio/vimms/blob/master/demo/01.%20Data/02.%20Extracting%20Chemicals%20from%20an%20mzML%20file.ipynb) to see how to generate this.

In [6]:
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','..','tests','fixtures'))
dataset = load_obj(os.path.join(data_dir, 'beer_compounds.p'))

### Run SWATH Controller

In [7]:
rt_range = [(0, 1440)]
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

In [8]:
min_mz = 0
max_mz = 1000
width = 100
overlap = 0

In [9]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset)
controller = SWATH(min_mz, max_mz, width, scan_overlap=overlap)

2023-07-17 13:32:01.531 | DEBUG    | vimms.Chemicals:__init__:477 - FastMemoryChems initialised


In [10]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

  0%|          | 0/1440 [00:00<?, ?it/s]

Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [11]:
set_log_level_debug()
mzml_filename = 'swath_controller.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

/Users/joewandy/anaconda3/envs/vimms/lib/python3.9/site-packages/psims/document.py:434: ReferentialIntegrityWarning:

No reference was found for 100000 in Spectrum

2023-07-17 13:32:21.535 | DEBUG    | vimms.Environment:write_mzML:247 - Created mzML file /Users/joewandy/Work/git/vimms/demo/02. Methods/results/swath_controller.mzML


Spectral deconvolution is required before the MS2 scans in the generated mzML file can be used for analysis. Tools like [MS-DIAL](http://prime.psc.riken.jp/compms/msdial/main.html) are often used for this. The SWATH controller can generates a config file that can be loaded by MS-DIAL to process the resulting mzML file above.

In [12]:
filename = os.path.join(out_dir, 'swath_config.txt')
controller.write_msdial_experiment_file(filename)